<a href="https://colab.research.google.com/github/pena-rodrigo/rl_cartpole/blob/main/cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##explore gym

import gym
import time
env = gym.make("CartPole-v1")
observation = env.reset()
for _ in range(1000):
  # env.render()
  # time.sleep(0.1)
  action = env.action_space.sample()
  observation, reward, done, info = env.step(action)
    
  if done:
    observation = env.reset()
env.close()

In [ ]:
info

{}

In [32]:
##put together actor class

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from dataclasses import dataclass
from typing import Any
from random import sample

@dataclass
class Sarsd:
  state: Any
  action: int
  reward: float
  next_state: Any
  done: bool


class DQNAgent():
  def __init__(self, model):
    self.model = model

  def get_actions(self, observations):
    # obs shape is (N, 4)
    q_vals = self.model(observations)
    # q_vals  shape (N, 2)

    return q_vals.max(-1)[1]

class Model(nn.Module):
  def __init__(self, obs_shape, num_actions):
    super(Model, self).__init__()
    assert len(obs_shape)==1, "This network only works for flat observations"
    self.obs_shape = obs_shape
    self.num_actions= num_actions
    self.net = torch.nn.Sequential(
        torch.nn.Linear(obs_shape[0], 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, num_actions)
    )
    self.opt = optim.Adam(self.net.parameters(), lr=0.0001)

  def forward(self, x):
    return self.net(x)






In [33]:
# class ReplayBuffer
# s, a, r, s

class ReplayBuffer:
  def __init__(self, buffer_size=100000):
    self.buffer_size=buffer_size
    self.buffer = []

  def insert(self, sars):
    self.buffer.append(sars)
    self.buffer = self.buffer[-self.buffer_size:]

  def sample(self, num_samples):
    assert num_samples <= len(self.buffer)
    return sample(self.buffer, num_samples)

In [37]:
def update_tgt_model(m,tgt):
  tgt.load_state_dict(m.state_dict())

def train_step(model, state_transitions,tgt,num_actions):
  cur_states = torch.stack(([torch.Tensor(s.state) for s in state_transitions]))
  rewards = torch.stack(([torch.Tensor([s.reward]) for s in state_transitions]))
  mask = torch.stack(([torch.Tensor([0]) if s.done else torch.Tensor([1]) for s in state_transitions]))
  next_states = torch.stack(([torch.Tensor(s.next_state) for s in state_transitions]))
  actions = [s.action for s in state_transitions]

  with torch.no_grad():
    qvals_next = tgt(next_states).max(-1)[0] # (N, num_actions)

  model.opt.zero_grad()

  qvals = model(cur_states)
  one_hot_actions = F.one_hot(torch.LongTensor(actions), num_actions)
  loss = (rewards + mask[:,0]*qvals_next - torch.sum(qvals*one_hot_actions, -1)).mean()
  loss.backward()
  model.opt.step()
  return loss

env = gym.make("CartPole-v1")
last_observation = env.reset()

m = Model(env.observation_space.shape, env.action_space.n)
tgt = Model(env.observation_space.shape, env.action_space.n)

rb = ReplayBuffer()

# qvals = m(torch.Tensor(observation))

try:
  while True:
    action = env.action_space.sample() #your agent takes random actions
    observation, reward, done, info = env.step(action)

    rb.insert(Sarsd(last_observation, action, reward, observation, done))
    last_observation = observation

    if done:
      observation = env.reset()

    if len(rb.buffer)> 5000:
      loss = train_step(m, rb.sample(1000), tgt, env.action_space.n)
      print(loss)
      raise Exception()
    #   import ipdb; ipdb.set_trace()
except KeyboardInterrupt:
  pass
env.close()


# for _ in range(1000):
  # env.render()
  # time.sleep(0.1)
  # action = env.action_space.sample()
  # observation, reward, done, info = env.step(action)
    

# 

tensor(1.3283, grad_fn=<MeanBackward0>)


Exception: ignored

In [29]:
rewards

NameError: ignored

In [16]:
train_step(m, rb.sample(5000), tft, 2)

NameError: ignored